# Data Cleaning
---


**Objective:** to get the data for the following information for all countries from December 1959 to December 1990


1. Industrial production (Index)

2. Exchange rates, National Currency per US dollar (Period Average)

3. Consumer prices (All items), index

4. International Reserves and Liquidity (Reserves, Official Reserve Assets, US Dollar)

5. Data for consumer prices and international reserves for the United States only over the same time period.


---

# 1. Downloading the data

We collected the data from ['IMF data portal'](https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sid=1390030341854) using the query function to get desired data

the desired data for Germany and the USA can be found in 2 seperate excel files in the data folder of the repository, titled Germany and the USA respectively



---
# 2. Cleaning the data

#### Importing and merging the 2 datasets

In [ ]:
import pandas as pd
import warnings

# Suppress FutureWarning (if you prefer to not see them)
warnings.simplefilter("ignore", FutureWarning)

# Define the desired final column names (for 5 data columns)
final_columns = [
    "Time (Year/Month)",
    "Economic Activity, Industrial Production, Index",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate",
    "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar",
    "Prices, Consumer Price Index, All items, Index"
]

# --- Process Germany File ---
# Assume Germany.xlsx has the header row in Excel row 3 (so skip the first 2 rows) 
# and the first 5 columns are the ones we need.
germany_df = pd.read_excel("../data/Germany.xlsx", header=0, skiprows=2, usecols=[0,1,2,3,4])
germany_df.columns = final_columns
germany_df["Country"] = "Germany"

# --- Process USA File ---
# Assume USA.xlsx also uses row 3 as header (skip first 2 rows),
# but the file only has 3 columns: Time, International Reserves, and Prices.
usa_df = pd.read_excel("../data/USA.xlsx", header=0, skiprows=2)
# Select the first 3 columns (if there are extra columns, adjust this accordingly)
usa_df = usa_df.iloc[:, :3]

# Rename the existing columns according to what they actually represent in the USA file.
usa_df.columns = [
    "Time (Year/Month)",
    "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar",
    "Prices, Consumer Price Index, All items, Index"
]

# Insert two blank columns for the missing data:
# We need to insert "Economic Activity, Industrial Production, Index" at position 1
usa_df.insert(loc=1, column="Economic Activity, Industrial Production, Index", value=pd.NA)
# And insert "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate" at position 2
usa_df.insert(loc=2, column="Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate", value=pd.NA)

# Now the USA DataFrame has the same 5 columns as defined in final_columns.
usa_df["Country"] = "USA"

# --- Merge the Two DataFrames ---
merged_df = pd.concat([germany_df, usa_df], ignore_index=True)

merged_df 


---

## 3. Construction of variables for Germany:

#### 3.1 The monthly growth in the nominal exchange rate for Germany: 

I calculated this variable by simply using the inbuilt pct_change function and multiplying the result by 100

In [ ]:
import pandas as pd

# Assume merged_df is already created from previous steps

# Filter for Germany and work on a copy
germany_data = merged_df[merged_df["Country"] == "Germany"].copy()

# Convert the "Time (Year/Month)" column to datetime (assuming format like "Dec 1959")
germany_data["Time (Year/Month)"] = pd.to_datetime(germany_data["Time (Year/Month)"], format='%b %Y')

# Sort the data chronologically
germany_data.sort_values("Time (Year/Month)", inplace=True)

# Calculate monthly percentage change for the nominal exchange rate and multiply by 100 to express it as a percentage
germany_data["german_monthly_nominal_exchange_rate_growth"] = (
    germany_data["Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate"].pct_change() * 100
)

# Create a new DataFrame with only the Time and computed growth columns
german_monthly_change_in_nominal_exchange_rate = germany_data[[
    "Time (Year/Month)", 
    "german_monthly_nominal_exchange_rate_growth"
]].copy()

# (Optional) Reset the index for cleanliness
german_monthly_change_in_nominal_exchange_rate.reset_index(drop=True, inplace=True)

# Now, german_change_in_nominal_exchange_rate contains only the desired two columns
german_monthly_change_in_nominal_exchange_rate


,Time (Year/Month),german_monthly_nominal_exchange_rate_growth
0,1959-12-01,NaN
1,1960-01-01,0.000000
2,1960-02-01,0.000000
3,1960-03-01,0.000000
4,1960-04-01,0.000000
...,...,...
368,1990-08-01,-4.219769
369,1990-09-01,-0.063666
370,1990-10-01,-2.955979
371,1990-11-01,-2.382984


#### 3.2 The monthly growth in the real exchange rate

In [ ]:
import pandas as pd
import numpy as np

# ---------------------------
# 1) Filter and prepare Germany data
# ---------------------------
# Assume 'merged_df' is the DataFrame that contains:
#   - "Country" column
#   - "Time (Year/Month)" (e.g. "Dec 1959")
#   - "Prices, Consumer Price Index, All items, Index" (Germany's CPI)
#   - "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate" (Germany's nominal exchange rate)
# We'll extract and rename columns for clarity.

germany_data = merged_df.loc[merged_df["Country"] == "Germany", [
    "Time (Year/Month)",
    "Prices, Consumer Price Index, All items, Index",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate"
]].copy()

# Convert the time column to datetime (format like "Dec 1959")
germany_data["Time (Year/Month)"] = pd.to_datetime(
    germany_data["Time (Year/Month)"], format='%b %Y'
)

# Rename columns for convenience
germany_data.rename(columns={
    "Prices, Consumer Price Index, All items, Index": "CPI_GER",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "EXCH_GER"
}, inplace=True)

# Sort by time
germany_data.sort_values("Time (Year/Month)", inplace=True)

# Calculate monthly log changes:
#   inflation_ger = ln(CPI_GER_t) - ln(CPI_GER_{t-1})
germany_data["inflation_ger"] = np.log(germany_data["CPI_GER"]).diff()

#   nominal_exch_rate_growth = ln(EXCH_GER_t) - ln(EXCH_GER_{t-1})
germany_data["nominal_exch_rate_growth"] = np.log(germany_data["EXCH_GER"]).diff()

# ---------------------------
# 2) Filter and prepare US data
# ---------------------------
# We assume the US rows have a column "Prices, Consumer Price Index, All items, Index" for the US CPI.
us_data = merged_df.loc[merged_df["Country"] == "USA", [
    "Time (Year/Month)",
    "Prices, Consumer Price Index, All items, Index"
]].copy()

us_data["Time (Year/Month)"] = pd.to_datetime(
    us_data["Time (Year/Month)"], format='%b %Y'
)

us_data.rename(columns={
    "Prices, Consumer Price Index, All items, Index": "CPI_US"
}, inplace=True)

us_data.sort_values("Time (Year/Month)", inplace=True)

# Calculate monthly log changes for US inflation
us_data["inflation_us"] = np.log(us_data["CPI_US"]).diff()

# ---------------------------
# 3) Merge Germany with US data by month
# ---------------------------
merged_ger_us = pd.merge(
    germany_data,
    us_data[["Time (Year/Month)", "inflation_us"]],  # only need US inflation
    on="Time (Year/Month)",
    how="inner"  # or 'left' if you want all Germany months even if US is missing
)

# ---------------------------
# 4) Compute monthly real exchange rate growth
# ---------------------------
# Using the approximate log-change formula:
#   real_exch_rate_growth = nominal_exch_rate_growth + inflation_ger - inflation_us
merged_ger_us["real_exch_rate_growth"] = (
    merged_ger_us["nominal_exch_rate_growth"] 
    + merged_ger_us["inflation_ger"] 
    - merged_ger_us["inflation_us"]
)

# Multiply by 100 to get percentage terms:
merged_ger_us["real_exch_rate_growth"] *= 100

# ---------------------------
# 5) Create a final DataFrame with only the time and real exchange rate growth
# ---------------------------
german_change_in_real_exchange_rate = merged_ger_us[[
    "Time (Year/Month)", 
    "real_exch_rate_growth"
]].copy()

# 6.)  Reset the index for neatness
german_change_in_real_exchange_rate.reset_index(drop=True, inplace=True)


german_change_in_real_exchange_rate


,Time (Year/Month),real_exch_rate_growth
0,1959-12-01,NaN
1,1960-01-01,0.340716
2,1960-02-01,-0.910483
3,1960-03-01,0.000000
4,1960-04-01,0.230208
...,...,...
368,1990-08-01,-4.916377
369,1990-09-01,-0.585995
370,1990-10-01,-2.881808
371,1990-11-01,-2.841436


#### 3.3 An index of the real exchange rate (setting the real exchange rate for December 1990 = 1)


In [ ]:
import pandas as pd
import numpy as np

# ---------------------------
# 1) Filter and prepare Germany data
# ---------------------------
germany_data = merged_df.loc[
    merged_df["Country"] == "Germany",
    ["Time (Year/Month)",
     "Prices, Consumer Price Index, All items, Index",
     "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate"]
].copy()

# Convert the date strings (like "Dec 1990") to actual datetime objects
germany_data["Time (Year/Month)"] = pd.to_datetime(
    germany_data["Time (Year/Month)"], format='%b %Y'
)

# Rename columns for clarity
germany_data.rename(columns={
    "Prices, Consumer Price Index, All items, Index": "CPI_GER",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "EXCH_GER"
}, inplace=True)

# Sort chronologically
germany_data.sort_values("Time (Year/Month)", inplace=True)

# ---------------------------
# 2) Filter and prepare US data
# ---------------------------
us_data = merged_df.loc[
    merged_df["Country"] == "USA",
    ["Time (Year/Month)",
     "Prices, Consumer Price Index, All items, Index"]
].copy()

us_data["Time (Year/Month)"] = pd.to_datetime(
    us_data["Time (Year/Month)"], format='%b %Y'
)
us_data.rename(columns={
    "Prices, Consumer Price Index, All items, Index": "CPI_US"
}, inplace=True)

us_data.sort_values("Time (Year/Month)", inplace=True)

# ---------------------------
# 3) Merge the two DataFrames by month
# ---------------------------
ger_us_merged = pd.merge(
    germany_data,
    us_data[["Time (Year/Month)", "CPI_US"]],
    on="Time (Year/Month)",
    how="inner"
)

# ---------------------------
# 4) Compute the Real Exchange Rate
# ---------------------------
# RER_t = EXCH_GER_t * (CPI_US_t / CPI_GER_t)
ger_us_merged["real_exchange_rate"] = (
    ger_us_merged["EXCH_GER"] * (ger_us_merged["CPI_US"] / ger_us_merged["CPI_GER"])
)

# ---------------------------
# 5) Normalize so that December 1990 = 1
# ---------------------------
base_date = pd.to_datetime("Dec 1990", format='%b %Y')
base_row = ger_us_merged.loc[ger_us_merged["Time (Year/Month)"] == base_date, "real_exchange_rate"]

if len(base_row) == 0:
    raise ValueError("No data found for December 1990 in Germany's real exchange rate data.")

base_value = base_row.values[0]

# real_exchange_rate_index = real_exchange_rate / base_value
ger_us_merged["real_exchange_rate_index"] = ger_us_merged["real_exchange_rate"] / base_value

# ---------------------------
# 6) Create a final DataFrame with only time + the index
# ---------------------------
german_real_exchange_rate_index = ger_us_merged[[
    "Time (Year/Month)",
    "real_exchange_rate_index"
]].copy()

german_real_exchange_rate_index.reset_index(drop=True, inplace=True)


german_real_exchange_rate_index


,Time (Year/Month),real_exchange_rate_index
0,1959-12-01,1.713301
1,1960-01-01,1.707474
2,1960-02-01,1.723091
3,1960-03-01,1.723091
4,1960-04-01,1.719129
...,...,...
368,1990-08-01,1.045085
369,1990-09-01,1.049889
370,1990-10-01,1.017646
371,1990-11-01,0.997672


#### 3.4 German monthly inflation rate

In [ ]:
import pandas as pd

# --- Filter for Germany's CPI Data ---
germany_data = merged_df.loc[
    merged_df["Country"] == "Germany",
    ["Time (Year/Month)", "Prices, Consumer Price Index, All items, Index"]
].copy()

# --- Convert Time Column to Datetime ---
# Here, we assume the time strings are like "Dec 1990"
germany_data["Time (Year/Month)"] = pd.to_datetime(
    germany_data["Time (Year/Month)"], format='%b %Y'
)

# --- Sort the DataFrame Chronologically ---
germany_data.sort_values("Time (Year/Month)", inplace=True)

# --- Compute the Monthly Inflation Rate (Arithmetic Change) ---
# The inflation rate is computed as: ((CPI_t - CPI_{t-1}) / CPI_{t-1}) * 100
germany_data["monthly_inflation_rate"] = (
    (germany_data["Prices, Consumer Price Index, All items, Index"].diff() /
     germany_data["Prices, Consumer Price Index, All items, Index"].shift(1)) * 100
)

# --- Create a Final DataFrame with Only Time and Inflation Rate ---
german_monthly_inflation = germany_data[[
    "Time (Year/Month)", 
    "monthly_inflation_rate"
]].copy()

# Reset index for neatness
german_monthly_inflation.reset_index(drop=True, inplace=True)

german_monthly_inflation


,Time (Year/Month),monthly_inflation_rate
0,1959-12-01,NaN
1,1960-01-01,0.000000
2,1960-02-01,-0.568147
3,1960-03-01,0.000000
4,1960-04-01,0.571393
...,...,...
368,1990-08-01,0.311532
369,1990-09-01,0.310565
370,1990-10-01,0.722391
371,1990-11-01,-0.204922


#### 3.5 The monthly growth in industrial production

In [ ]:
import pandas as pd

# --- Filter for Germany's Industrial Production Data ---
# We assume that the merged_df DataFrame contains a column for the industrial production index
# labeled "Economic Activity, Industrial Production, Index".
germany_industrial = merged_df.loc[
    merged_df["Country"] == "Germany",
    ["Time (Year/Month)", "Economic Activity, Industrial Production, Index"]
].copy()

# --- Convert the "Time (Year/Month)" Column to Datetime ---
# Here we assume the time is formatted like "Dec 1990"
germany_industrial["Time (Year/Month)"] = pd.to_datetime(
    germany_industrial["Time (Year/Month)"], format='%b %Y'
)

# --- Sort the DataFrame Chronologically ---
germany_industrial.sort_values("Time (Year/Month)", inplace=True)

# --- Calculate the Monthly Growth in Industrial Production ---
# Compute the percentage change from one month to the next using arithmetic change.
# Formula: ((IP_t - IP_{t-1}) / IP_{t-1}) * 100
germany_industrial["german_monthly_industrial_production_growth"] = (
    (germany_industrial["Economic Activity, Industrial Production, Index"].diff() /
     germany_industrial["Economic Activity, Industrial Production, Index"].shift(1)) * 100
)

# --- Create a Final DataFrame with Only the Time and the Growth Rate ---
german_monthly_industrial_production_growth = germany_industrial[[
    "Time (Year/Month)",
    "german_monthly_industrial_production_growth"
]].copy()

german_monthly_industrial_production_growth.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
german_monthly_industrial_production_growth


,Time (Year/Month),german_monthly_industrial_production_growth
0,1959-12-01,NaN
1,1960-01-01,-4.019729
2,1960-02-01,-0.488181
3,1960-03-01,3.743868
4,1960-04-01,6.470881
...,...,...
368,1990-08-01,-3.590683
369,1990-09-01,13.863684
370,1990-10-01,7.107116
371,1990-11-01,-3.434553


#### 3.6 The annual growth in industrial production

In [ ]:
import pandas as pd

# --- Step 1: Filter and prepare Germany's Industrial Production Data ---
# We assume 'merged_df' (from previous steps) contains the column:
# "Economic Activity, Industrial Production, Index"
germany_industrial = merged_df.loc[
    merged_df["Country"] == "Germany",
    ["Time (Year/Month)", "Economic Activity, Industrial Production, Index"]
].copy()

# Convert the "Time (Year/Month)" column from strings (e.g. "Dec 1990") to datetime.
germany_industrial["Time (Year/Month)"] = pd.to_datetime(
    germany_industrial["Time (Year/Month)"], format='%b %Y'
)

# Sort the data by time to ensure the calculations are in order.
germany_industrial.sort_values("Time (Year/Month)", inplace=True)

# --- Step 2: Compute the Annual Industrial Production Growth ---
# For each month, the annual growth is calculated as:
#    ((IP_t - IP_{t-12}) / IP_{t-12}) * 100
# For the first 12 months, this will result in missing values.
germany_industrial["annual_growth"] = (
    (germany_industrial["Economic Activity, Industrial Production, Index"] -
     germany_industrial["Economic Activity, Industrial Production, Index"].shift(12))
    / germany_industrial["Economic Activity, Industrial Production, Index"].shift(12)
) * 100

# --- Step 3: Fill Gaps via Interpolation ---
# Replace any missing values (including the initial 12 values and any gaps due to outliers or missing data)
# with a linear interpolation. 'limit_direction="both"' ensures that missing values at the start and end are also filled.
germany_industrial["annual_growth_filled"] = germany_industrial["annual_growth"].interpolate(method='linear', limit_direction='both')

# --- Step 4: Create the Final DataFrame ---
# We extract only the "Time (Year/Month)" and the gap-filled annual growth rate,
# and rename the growth column to "german_annual_industrial_production_growth".
german_annual_industrial_production_growth = germany_industrial[[
    "Time (Year/Month)", "annual_growth_filled"
]].copy()

german_annual_industrial_production_growth.rename(
    columns={"annual_growth_filled": "german_annual_industrial_production_growth"},
    inplace=True
)

# Reset the index for neatness.
german_annual_industrial_production_growth.reset_index(drop=True, inplace=True)

german_annual_industrial_production_growth

,Time (Year/Month),german_annual_industrial_production_growth
0,1959-12-01,10.900123
1,1960-01-01,10.900123
2,1960-02-01,10.900123
3,1960-03-01,10.900123
4,1960-04-01,10.900123
...,...,...
368,1990-08-01,6.017897
369,1990-09-01,5.514275
370,1990-10-01,6.152998
371,1990-11-01,5.580779


#### 3.7 An index of the value of international reserves (value of reserves for January 1960 = 100)

In [ ]:
import pandas as pd

# --- Step 1: Filter for Germany's International Reserves Data ---
# We assume merged_df has the column "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"
germany_reserves = merged_df.loc[
    merged_df["Country"] == "Germany",
    ["Time (Year/Month)", "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"]
].copy()

# --- Step 2: Convert the Time Column to Datetime ---
# Assuming time strings like "Jan 1960", we use format '%b %Y'
germany_reserves["Time (Year/Month)"] = pd.to_datetime(
    germany_reserves["Time (Year/Month)"], format='%b %Y'
)

# --- Step 3: Sort Data Chronologically ---
germany_reserves.sort_values("Time (Year/Month)", inplace=True)

# --- Step 4: Compute the Real Reserves Index ---
# We set the base period as January 1960, where the index is defined to be 100.
base_date = pd.to_datetime("Jan 1960", format='%b %Y')
base_value_series = germany_reserves.loc[
    germany_reserves["Time (Year/Month)"] == base_date,
    "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"
]

if base_value_series.empty:
    raise ValueError("No data found for January 1960 in the international reserves column.")

base_value = base_value_series.iloc[0]

# Create the index: for each month, index = (current value / base value) * 100.
germany_reserves["german_international_reserves_index"] = (
    germany_reserves["International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"] / base_value
) * 100

# --- Step 5: Create a Final DataFrame with Only the Time and the Computed Index ---
german_international_reserves_index = germany_reserves[[
    "Time (Year/Month)",
    "german_international_reserves_index"
]].copy()

german_international_reserves_index.reset_index(drop=True, inplace=True)

german_international_reserves_index


,Time (Year/Month),german_international_reserves_index
0,1959-12-01,101.848342
1,1960-01-01,100.000000
2,1960-02-01,101.740143
3,1960-03-01,105.128964
4,1960-04-01,110.837171
...,...,...
368,1990-08-01,1533.093782
369,1990-09-01,1549.431834
370,1990-10-01,1587.837706
371,1990-11-01,1612.268481


---


## Variables for the USA

#### 3.8 The US monthly inflation rate

In [ ]:
import pandas as pd

# --- Filter for US CPI Data ---
# Extract only the columns for time and US CPI from the merged DataFrame.
us_data = merged_df.loc[
    merged_df["Country"] == "USA",
    ["Time (Year/Month)", "Prices, Consumer Price Index, All items, Index"]
].copy()

# --- Convert the "Time (Year/Month)" Column to Datetime ---
# Here, we assume the time strings are formatted like "Dec 1959".
us_data["Time (Year/Month)"] = pd.to_datetime(us_data["Time (Year/Month)"], format='%b %Y')

# --- Sort the DataFrame Chronologically ---
us_data.sort_values("Time (Year/Month)", inplace=True)

# --- Calculate the Monthly Inflation Rate (Arithmetic Change) ---
# The formula used is:
#    Inflation_t = ((CPI_t - CPI_{t-1}) / CPI_{t-1}) * 100
us_data["monthly_inflation_rate"] = (
    (us_data["Prices, Consumer Price Index, All items, Index"].diff() / 
     us_data["Prices, Consumer Price Index, All items, Index"].shift(1)) * 100
)

# --- Create a Final DataFrame with Only the Time and the Computed Inflation Rate ---
us_monthly_inflation = us_data[[
    "Time (Year/Month)",
    "monthly_inflation_rate"
]].copy()

# Optionally, reset the index for neatness
us_monthly_inflation.reset_index(drop=True, inplace=True)

us_monthly_inflation


,Time (Year/Month),monthly_inflation_rate
0,1959-12-01,NaN
1,1960-01-01,-0.340136
2,1960-02-01,0.341297
3,1960-03-01,0.000000
4,1960-04-01,0.340136
...,...,...
368,1990-08-01,0.920245
369,1990-09-01,0.835866
370,1990-10-01,0.602864
371,1990-11-01,0.224719


#### 3.9 An index of the value of international reserves (value of reserves for January 1960 = 100)

In [ ]:
import pandas as pd

# --- Step 1: Filter for US International Reserves Data ---
# We assume merged_df has the column "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"
us_reserves = merged_df.loc[
    merged_df["Country"] == "USA",
    ["Time (Year/Month)", "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"]
].copy()

# --- Step 2: Convert the Time Column to Datetime ---
# Assuming time strings like "Jan 1960", we use format '%b %Y'
us_reserves["Time (Year/Month)"] = pd.to_datetime(us_reserves["Time (Year/Month)"], format='%b %Y')

# --- Step 3: Sort the DataFrame Chronologically ---
us_reserves.sort_values("Time (Year/Month)", inplace=True)

# --- Step 4: Find the Base Value for January 1960 ---
base_date = pd.to_datetime("Jan 1960", format='%b %Y')
base_value_series = us_reserves.loc[
    us_reserves["Time (Year/Month)"] == base_date, 
    "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"
]

if base_value_series.empty:
    raise ValueError("No data found for January 1960 in the US international reserves column.")

base_value = base_value_series.iloc[0]

# --- Step 5: Compute the US International Reserves Index ---
# For each month, index = (current reserves / base value) * 100.
us_reserves["us_international_reserves_index"] = (
    us_reserves["International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar"] / base_value
) * 100

# --- Step 6: Create a Final DataFrame with Only Time and the Computed Index ---
us_international_reserves_index = us_reserves[[
    "Time (Year/Month)", "us_international_reserves_index"
]].copy()

us_international_reserves_index.reset_index(drop=True, inplace=True)

us_international_reserves_index


,Time (Year/Month),us_international_reserves_index
0,1959-12-01,100.122916
1,1960-01-01,100.000000
2,1960-02-01,99.616353
3,1960-03-01,99.378902
4,1960-04-01,99.068353
...,...,...
368,1990-08-01,367.392080
369,1990-09-01,372.584941
370,1990-10-01,385.751982
371,1990-11-01,386.716715


---
# 4. correction of outliers and missing values

I am going to be identifying outliers by trying to find values greater/lower than 3 standard deviations from the mean

#### Function to Replace missing values (inlcuding outliers) using interpolation

In [ ]:
import pandas as pd
import numpy as np

def clean_columns(df, columns, sd=4):
    """
    For each column in the provided list, remove outliers that fall outside ±(sd * std) of the mean 
    (i.e., set them to NaN), then replace missing values using linear interpolation.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to clean.
        columns (list of str): The list of column names in df to clean.
        sd (float): The number of standard deviations to use as the cutoff (default is 4).
    
    Returns:
        pd.DataFrame: A copy of df with the specified columns cleaned.
    """
    df_clean = df.copy()
    for col in columns:
        if col not in df_clean.columns:
            print(f"Column '{col}' not found in DataFrame. Skipping.")
            continue
        
        mean_val = df_clean[col].mean()
        std_val = df_clean[col].std()
        lower_bound = mean_val - sd * std_val
        upper_bound = mean_val + sd * std_val
        
        # Replace outliers with NaN
        df_clean.loc[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound), col] = np.nan
        
        # Fill missing values by linear interpolation in both directions.
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='both')
    return df_clean




Using this function to correcting outliers and missing values for collected (not constructed) base variables

In [ ]:
# Assuming merged_df has a "Country" column with values "Germany" or "USA"
g_df = merged_df[merged_df["Country"] == "Germany"].copy()
us_df = merged_df[merged_df["Country"] == "USA"].copy()

import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def clean_columns_firstrow_from_right(df, columns, sd=4):
    """
    For each column in the provided list:
      1. Remove outliers outside ±(sd*std) from the mean by setting them to NaN.
      2. Sort the DataFrame by "Time (Year/Month)" to ensure chronological order.
      3. Fill the very first row by interpolating from the next valid row (limit_direction="backward").
      4. Fill any remaining missing values by two-sided linear interpolation (limit_direction="both").

    Parameters:
      df (pd.DataFrame): The DataFrame to clean. Must contain "Time (Year/Month)" for sorting.
      columns (list of str): List of column names in df to clean.
      sd (float): Number of standard deviations for the outlier cutoff (default is 4).

    Returns:
      pd.DataFrame: A copy of df with the specified columns cleaned.
    """
    # Work on a copy so we don't modify the original
    df_clean = df.copy()

    # Sort by time so the first row is truly the earliest
    if "Time (Year/Month)" in df_clean.columns:
        df_clean = df_clean.sort_values("Time (Year/Month)").reset_index(drop=True)
    else:
        print("Warning: No 'Time (Year/Month)' column found. The first-row interpolation logic may not work as intended.")

    for col in columns:
        if col not in df_clean.columns:
            print(f"Column '{col}' not found in DataFrame. Skipping column {col}.")
            continue
        
        # 1. Identify outliers
        mean_val = df_clean[col].mean()
        std_val = df_clean[col].std()
        lower_bound = mean_val - sd * std_val
        upper_bound = mean_val + sd * std_val
        
        # Replace outliers with NaN
        outlier_mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        df_clean.loc[outlier_mask, col] = np.nan
        
        # 2. Fill the first row from the right if it's missing
        # (by doing a one-sided interpolation backward in ascending time)
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='backward')
        
        # 3. Fill the rest by normal two-sided interpolation
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='both')
    
    return df_clean


using the clean columns function to correct outliers and missing values for constructed variables

In [ ]:

# -----------------------------------------------------
# Define a dictionary mapping each original DataFrame (object)
# to the column name that should be cleaned.
#
# The keys below are the variable names you have in your environment.
# The value in each tuple is the column name that contains the numeric data.
# -----------------------------------------------------
dfs_to_clean = {
    "german_monthly_change_in_nominal_exchange_rate": (german_monthly_change_in_nominal_exchange_rate, "german_monthly_nominal_exchange_rate_growth"),
    "german_change_in_real_exchange_rate": (german_change_in_real_exchange_rate, "real_exch_rate_growth"),
    "german_real_exchange_rate_index": (german_real_exchange_rate_index, "real_exchange_rate_index"),
    "german_monthly_inflation": (german_monthly_inflation, "monthly_inflation_rate"),
    "german_monthly_industrial_production_growth": (german_monthly_industrial_production_growth, "german_monthly_industrial_production_growth"),
    "german_annual_industrial_production_growth": (german_annual_industrial_production_growth, "german_annual_industrial_production_growth"),
    "german_international_reserves_index": (german_international_reserves_index, "german_international_reserves_index"),
    "us_monthly_inflation": (us_monthly_inflation, "monthly_inflation_rate"),
    "us_international_reserves_index": (us_international_reserves_index, "us_international_reserves_index")
}

# -----------------------------------------------------
# Process each DataFrame using the clean_columns function
# and store the result in a new dictionary with keys prefixed by "cleaned_"
# -----------------------------------------------------
cleaned_dfs = {}
for df_name, (df_obj, col_name) in dfs_to_clean.items():
    # Optionally, ensure "Time (Year/Month)" is converted to datetime:
    if "Time (Year/Month)" in df_obj.columns:
        df_obj["Time (Year/Month)"] = pd.to_datetime(df_obj["Time (Year/Month)"], errors="coerce")
    # Clean the specified column (as a one-element list)
    cleaned_df_obj = clean_columns(df_obj, [col_name], sd=4)
    # Store in the dictionary with a new key
    cleaned_dfs["cleaned_" + df_name] = cleaned_df_obj

# Now, cleaned_dfs contains cleaned versions of your 9 DataFrames.
# You can inspect one, for example:
cleaned_dfs["cleaned_german_monthly_change_in_nominal_exchange_rate"]


,Time (Year/Month),german_monthly_nominal_exchange_rate_growth
0,1959-12-01,0.000000
1,1960-01-01,0.000000
2,1960-02-01,0.000000
3,1960-03-01,0.000000
4,1960-04-01,0.000000
...,...,...
368,1990-08-01,-4.219769
369,1990-09-01,-0.063666
370,1990-10-01,-2.955979
371,1990-11-01,-2.382984


Assembling final DF

reassigning country names

In [ ]:
# Reassigning country for easier merging:

# Loop over the cleaned_dfs dictionary and add/update the "Country" column
for key, df in cleaned_dfs.items():
    if key.startswith("cleaned_german_"):
        df["Country"] = "Germany"
    elif key.startswith("cleaned_us_"):
        df["Country"] = "USA"
    else:
        # Optionally, handle unexpected keys
        df["Country"] = "Unknown"








obtaining new german df

In [ ]:
import pandas as pd
import functools

# Step 1: Get the 7 cleaned German constructed DataFrames from cleaned_dfs.
# (Assuming your dictionary "cleaned_dfs" contains keys starting with "cleaned_german_"
# for all German constructed variables.)
german_constructed = [df for key, df in cleaned_dfs.items() if key.startswith("cleaned_german_")]

# Check that we have 7:
print("Number of German constructed DataFrames:", len(german_constructed))

# Step 2: Merge the 7 constructed DataFrames into one wide DataFrame.
# Use an outer join on "Time (Year/Month)" and "Country".
if german_constructed:
    merged_german_constructed = functools.reduce(
        lambda left, right: pd.merge(left, right, on=["Time (Year/Month)", "Country"], how="outer"),
        german_constructed
    )
else:
    merged_german_constructed = pd.DataFrame()

# Ensure the "Time (Year/Month)" column in both DataFrames is in datetime format
g_df["Time (Year/Month)"] = pd.to_datetime(g_df["Time (Year/Month)"], errors="coerce")
merged_german_constructed["Time (Year/Month)"] = pd.to_datetime(merged_german_constructed["Time (Year/Month)"], errors="coerce")

# Merge the base German data (g_df) with the merged constructed German variables
new_g_df = pd.merge(g_df, merged_german_constructed, on=["Time (Year/Month)", "Country"], how="outer")

# Sort the resulting DataFrame by "Time (Year/Month)"
new_g_df = new_g_df.sort_values("Time (Year/Month)").reset_index(drop=True)

# For viewing:
new_g_df


# Step 3: Merge the base German data (g_df) with the merged German constructed data.
# We assume g_df contains the German base variables and already has columns "Time (Year/Month)" and "Country".
new_g_df = pd.merge(g_df, merged_german_constructed, on=["Time (Year/Month)", "Country"], how="outer")

# Optionally sort the final German DataFrame by time.
final_german_df = new_g_df.sort_values("Time (Year/Month)").reset_index(drop=True)

# Display the new German DataFrame

final_german_df


Number of German constructed DataFrames: 7


C:\Users\jogim\AppData\Local\Temp\ipykernel_20192\4071667737.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  g_df["Time (Year/Month)"] = pd.to_datetime(g_df["Time (Year/Month)"], errors="coerce")


,Time (Year/Month),"Economic Activity, Industrial Production, Index","Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate","International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index",Country,german_monthly_nominal_exchange_rate_growth,real_exch_rate_growth,real_exchange_rate_index,monthly_inflation_rate,german_monthly_industrial_production_growth,german_annual_industrial_production_growth,german_international_reserves_index
0,1959-12-01,32.500305,4.2000,4811.474341,24.616929,Germany,0.000000,0.340716,1.713301,0.000000,-4.019729,10.900123,101.848342
1,1960-01-01,31.193881,4.2000,4724.155785,24.616929,Germany,0.000000,0.340716,1.707474,0.000000,-4.019729,10.900123,100.000000
2,1960-02-01,31.041599,4.2000,4806.362830,24.477068,Germany,0.000000,-0.910483,1.723091,-0.568147,-0.488181,10.900123,101.740143
3,1960-03-01,32.203755,4.2000,4966.456016,24.477068,Germany,0.000000,0.000000,1.723091,0.000000,3.743868,10.900123,105.128964
4,1960-04-01,34.287622,4.2000,5236.120624,24.616929,Germany,0.000000,0.230208,1.719129,0.571393,6.470881,10.900123,110.837171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,1990-08-01,75.964955,1.5707,72425.738573,67.556702,Germany,-4.219769,-4.916377,1.045085,0.311532,-3.590683,6.017897,1533.093782
369,1990-09-01,86.496497,1.5697,73197.573621,67.766509,Germany,-0.063666,-0.585995,1.049889,0.310565,13.863684,5.514275,1549.431834
370,1990-10-01,92.643903,1.5233,75011.926830,68.256049,Germany,-2.955979,-2.881808,1.017646,0.722391,7.107116,6.152998,1587.837706
371,1990-11-01,89.461999,1.4870,76166.074709,68.116177,Germany,-2.382984,-2.841436,0.997672,-0.204922,-3.434553,5.580779,1612.268481


obtaining new USA df:

In [ ]:
import pandas as pd

# -------------------------------------------------
# Step 1: Ensure consistent "Country" and datetime in each US DataFrame
# -------------------------------------------------
us_df["Country"] = "USA"
us_df["Time (Year/Month)"] = pd.to_datetime(us_df["Time (Year/Month)"], errors="coerce")

us_monthly_inflation["Country"] = "USA"
us_monthly_inflation["Time (Year/Month)"] = pd.to_datetime(us_monthly_inflation["Time (Year/Month)"], errors="coerce")

us_international_reserves_index["Country"] = "USA"
us_international_reserves_index["Time (Year/Month)"] = pd.to_datetime(
    us_international_reserves_index["Time (Year/Month)"], errors="coerce"
)

# -------------------------------------------------
# Step 2: Merge the two constructed US DataFrames (inflation, reserves)
# -------------------------------------------------
us_constructed = pd.merge(
    us_monthly_inflation.drop_duplicates(subset=["Time (Year/Month)", "Country"]),
    us_international_reserves_index.drop_duplicates(subset=["Time (Year/Month)", "Country"]),
    on=["Time (Year/Month)", "Country"],
    how="outer"
)

# -------------------------------------------------
# Step 3: Merge the base US DataFrame with the constructed DataFrame
# -------------------------------------------------
new_us_df = pd.merge(
    us_df.drop_duplicates(subset=["Time (Year/Month)", "Country"]),
    us_constructed,
    on=["Time (Year/Month)", "Country"],
    how="outer"
)

# -------------------------------------------------
# Step 4: Sort, reset index, drop duplicates if needed
# -------------------------------------------------
new_us_df = new_us_df.sort_values("Time (Year/Month)").reset_index(drop=True)
new_us_df.drop_duplicates(subset=["Time (Year/Month)", "Country"], keep="last", inplace=True)

# -------------------------------------------------
# Step 5: Manually set the first row's monthly_inflation_rate to 0 if it's NaN
# -------------------------------------------------
if "monthly_inflation_rate" in new_us_df.columns and not new_us_df.empty:
    if pd.isna(new_us_df.loc[0, "monthly_inflation_rate"]):
        new_us_df.loc[0, "monthly_inflation_rate"] = 0

# new_us_df now contains the base US variables plus the constructed ones
new_us_df


C:\Users\jogim\AppData\Local\Temp\ipykernel_20192\2518442809.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  us_df["Time (Year/Month)"] = pd.to_datetime(us_df["Time (Year/Month)"], errors="coerce")


,Time (Year/Month),"Economic Activity, Industrial Production, Index","Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate","International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index",Country,monthly_inflation_rate,us_international_reserves_index
0,1959-12-01,NaN,NaN,21504.500000,13.482806,USA,0.000000,100.122916
1,1960-01-01,NaN,NaN,21478.100000,13.436946,USA,-0.340136,100.000000
2,1960-02-01,NaN,NaN,21395.700000,13.482806,USA,0.341297,99.616353
3,1960-03-01,NaN,NaN,21344.700000,13.482806,USA,0.000000,99.378902
4,1960-04-01,NaN,NaN,21278.000000,13.528666,USA,0.340136,99.068353
...,...,...,...,...,...,...,...,...
368,1990-08-01,NaN,NaN,78908.838357,60.351608,USA,0.920245,367.392080
369,1990-09-01,NaN,NaN,80024.166133,60.856066,USA,0.835866,372.584941
370,1990-10-01,NaN,NaN,82852.196532,61.222946,USA,0.602864,385.751982
371,1990-11-01,NaN,NaN,83059.402774,61.360525,USA,0.224719,386.716715


concatenate the df's to produce new merged dataframe:

In [ ]:
import pandas as pd

# 1) Concatenate the two DataFrames row-wise, using an outer join of columns
#    (this is the default for pd.concat with axis=0). 
#    For rows belonging to Germany, the US-only columns will be NaN.
#    For rows belonging to the US, the German-only columns will be NaN.
final_merged_df = pd.concat([final_german_df, new_us_df], ignore_index=True)

# 2) Sort by "Country" and "Time (Year/Month)" for clarity
final_merged_df = final_merged_df.sort_values(["Country", "Time (Year/Month)"]).reset_index(drop=True)

# 3) Now final_merged_df contains all the columns from both new_g_df and new_us_df.
#    For Germany rows, you’ll see all the German columns filled, US columns as NaN;
#    for US rows, the German columns will be NaN, and the US columns will be filled.

final_merged_df



,Time (Year/Month),"Economic Activity, Industrial Production, Index","Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate","International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index",Country,german_monthly_nominal_exchange_rate_growth,real_exch_rate_growth,real_exchange_rate_index,monthly_inflation_rate,german_monthly_industrial_production_growth,german_annual_industrial_production_growth,german_international_reserves_index,us_international_reserves_index
0,1959-12-01,32.500305,4.2,4811.474341,24.616929,Germany,0.0,0.340716,1.713301,0.000000,-4.019729,10.900123,101.848342,NaN
1,1960-01-01,31.193881,4.2,4724.155785,24.616929,Germany,0.0,0.340716,1.707474,0.000000,-4.019729,10.900123,100.000000,NaN
2,1960-02-01,31.041599,4.2,4806.362830,24.477068,Germany,0.0,-0.910483,1.723091,-0.568147,-0.488181,10.900123,101.740143,NaN
3,1960-03-01,32.203755,4.2,4966.456016,24.477068,Germany,0.0,0.000000,1.723091,0.000000,3.743868,10.900123,105.128964,NaN
4,1960-04-01,34.287622,4.2,5236.120624,24.616929,Germany,0.0,0.230208,1.719129,0.571393,6.470881,10.900123,110.837171,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,1990-08-01,NaN,NaN,78908.838357,60.351608,USA,NaN,NaN,NaN,0.920245,NaN,NaN,NaN,367.392080
742,1990-09-01,NaN,NaN,80024.166133,60.856066,USA,NaN,NaN,NaN,0.835866,NaN,NaN,NaN,372.584941
743,1990-10-01,NaN,NaN,82852.196532,61.222946,USA,NaN,NaN,NaN,0.602864,NaN,NaN,NaN,385.751982
744,1990-11-01,NaN,NaN,83059.402774,61.360525,USA,NaN,NaN,NaN,0.224719,NaN,NaN,NaN,386.716715


In [ ]:
import pandas as pd

# Make a copy of the cleaned merged DataFrame (which should have all constructed variables)
final_df = final_merged_df.copy()

# Rename columns for clarity
final_df = final_df.rename(columns={
    "Time (Year/Month)": "Time",
    "Economic Activity, Industrial Production, Index": "Industrial Production Index",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "Nominal Exchange Rate",
    "International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar": "International Reserves (USD)",
    "Prices, Consumer Price Index, All items, Index": "CPI",
    "german_monthly_nominal_exchange_rate_growth": "German Nominal EX Rate Growth",
    "real_exch_rate_growth": "German Real EX Rate Growth",
    "real_exchange_rate_index": "German Real EX Rate Index",
    "monthly_inflation_rate": "Inflation Rate",
    "german_monthly_industrial_production_growth": "German Monthly IP Growth",
    "german_annual_industrial_production_growth": "German Annual IP Growth",
    "german_international_reserves_index": "German International Reserves Index",
    "us_international_reserves_index": "US International Reserves Index"
})

# Combine the two International Reserves Index columns into one.
# This takes the German value if available; otherwise uses the US value.
final_df["International Reserves Index"] = final_df["German International Reserves Index"].combine_first(
    final_df["US International Reserves Index"]
)

# Optionally, if you prefer US values to override German values when available,
# you could swap the order:
# final_df["International Reserves Index"] = final_df["US International Reserves Index"].combine_first(
#     final_df["German International Reserves Index"]
# )

# Drop the original two columns as they are now merged into one.
final_df = final_df.drop(columns=["German International Reserves Index", "US International Reserves Index"])

# Export the final DataFrame to an Excel file called "final_df.xlsx" in the ../data/ folder.
final_df.to_excel("../data/final_df.xlsx", index=False)
final_df


,Time,Industrial Production Index,Nominal Exchange Rate,International Reserves (USD),CPI,Country,German Nominal EX Rate Growth,German Real EX Rate Growth,German Real EX Rate Index,Inflation Rate,German Monthly IP Growth,German Annual IP Growth,International Reserves Index
0,1959-12-01,32.500305,4.2,4811.474341,24.616929,Germany,0.0,0.340716,1.713301,0.000000,-4.019729,10.900123,101.848342
1,1960-01-01,31.193881,4.2,4724.155785,24.616929,Germany,0.0,0.340716,1.707474,0.000000,-4.019729,10.900123,100.000000
2,1960-02-01,31.041599,4.2,4806.362830,24.477068,Germany,0.0,-0.910483,1.723091,-0.568147,-0.488181,10.900123,101.740143
3,1960-03-01,32.203755,4.2,4966.456016,24.477068,Germany,0.0,0.000000,1.723091,0.000000,3.743868,10.900123,105.128964
4,1960-04-01,34.287622,4.2,5236.120624,24.616929,Germany,0.0,0.230208,1.719129,0.571393,6.470881,10.900123,110.837171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,1990-08-01,NaN,NaN,78908.838357,60.351608,USA,NaN,NaN,NaN,0.920245,NaN,NaN,367.392080
742,1990-09-01,NaN,NaN,80024.166133,60.856066,USA,NaN,NaN,NaN,0.835866,NaN,NaN,372.584941
743,1990-10-01,NaN,NaN,82852.196532,61.222946,USA,NaN,NaN,NaN,0.602864,NaN,NaN,385.751982
744,1990-11-01,NaN,NaN,83059.402774,61.360525,USA,NaN,NaN,NaN,0.224719,NaN,NaN,386.716715
